In [1]:
#importing all the required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from PIL import Image
import cv2
import time

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization, Input, MaxPooling2D
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications.vgg16 import VGG16,  preprocess_input
from tensorflow.keras.applications import VGG19, DenseNet121

from sklearn.metrics import classification_report, confusion_matrix
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [2]:
def custom_model():
    cus_model = VGG19(input_shape = (256,256,3),        
                         weights='imagenet', 
                         include_top= False,)
    
   #Using pre-trained weights from imagenet 
    for layer in cus_model.layers:
        layer.trainable = False

    # Adding layers in a sequential manner
    x = Flatten()(cus_model.output)

    x = Dense(512, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(5, activation='softmax')(x)

    model = Model(cus_model.input,x)
    return model

In [3]:
classes={0:'downdog',
 1:'goddess',
 2:'plank',
 3:'tree',
 4:'warrior2'}

In [4]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [5]:
def godess_feedback(landmarks):
    feed=''
    
    l_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
    l_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
    l_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
    
    r_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
    r_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
    r_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
    
    l_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
    l_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
    l_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
    
    r_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
    r_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
    r_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
       
    
    lh_angle=calculate_angle(l_shoulder,l_elbow,l_wrist)
    rh_angle=calculate_angle(r_shoulder,r_elbow,r_wrist)
    ll_angle=calculate_angle(l_hip,l_knee,l_ankle)
    rl_angle=calculate_angle(r_hip,r_knee,r_ankle)
    
    
    if 80<lh_angle<110:
        pass
    else:
        feed+='adjust left hand '
        
    if 80<rh_angle<110:
        pass
    else:
        feed+='adjust right hand '
        
    if 100<ll_angle<145:
        pass
    else:
        feed+='adjust left leg '
        
    if 100<rl_angle<145:
        pass
    else:
        feed+='adjust right leg '
        
    if 80<lh_angle<110 and 80<rh_angle<110 and 100<ll_angle<145 and 100<rl_angle<145:
        feed+='perfect'

    return feed

In [6]:
def downdog_feedback(landmarks):
    feed=''
    
    l_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
    l_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
    l_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
    
    r_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
    r_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
    r_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
    
    l_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
    l_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
    l_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
    
    r_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
    r_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
    r_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
       
    
    lh_angle=calculate_angle(l_shoulder,l_elbow,l_wrist)
    rh_angle=calculate_angle(r_shoulder,r_elbow,r_wrist)
    ll_angle=calculate_angle(l_hip,l_knee,l_ankle)
    rl_angle=calculate_angle(r_hip,r_knee,r_ankle)
    hip_angle= calculate_angle(l_shoulder,l_hip,l_knee)
    
    if lh_angle>160:
        pass
    else:
        feed+='adjust left hand '
        
    if rh_angle>160:
        pass
    else:
        feed+='adjust right hand '
        
    if ll_angle>160:
        pass
    else:
        feed+='adjust left leg '
        
    if rl_angle>160:
        pass
    else:
        feed+='adjust right leg '
        
    if 100>hip_angle>60:
        pass
    else:
        feed+='bend correctly'
        
    if lh_angle>160 and rh_angle>160 and ll_angle>160 and rl_angle>160 and 100>hip_angle>60: 
        feed+='perfect'
        
    return feed

In [7]:
def plank_feedback(landmarks):
    feed=''
    
    l_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
    l_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
    l_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
    
    r_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
    r_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
    r_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
    
    l_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
    l_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
    l_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
    
    r_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
    r_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
    r_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
       
    
    lh_angle=calculate_angle(l_shoulder,l_elbow,l_wrist)
    rh_angle=calculate_angle(r_shoulder,r_elbow,r_wrist)
    ll_angle=calculate_angle(l_hip,l_knee,l_ankle)
    rl_angle=calculate_angle(r_hip,r_knee,r_ankle)
    hip_angle= calculate_angle(l_shoulder,l_hip,l_knee)
    
    if 80<lh_angle<100:
        pass
    else:
        feed+='adjust left hand '
        
    if 80<rh_angle<100:
        pass
    else:
        feed+='adjust right hand '
        
    if 160<ll_angle<180:
        pass
    else:
        feed+='keep legs straight '
        
    if 160<rl_angle<180:
        pass
    else:
        feed+='keep legs straight '
        
    if 160<hip_angle<180:
        pass
    else:
        feed+='bend correctly'
        
    if 80<lh_angle<100 and 80<rh_angle<100 and 160<ll_angle<180 and 160<rl_angle<180 and 160<hip_angle<180:
        feed+='perfect'
        
    return feed

In [8]:
def tree_feedback(landmarks):
    feed=''
    c=0
    
    l_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
    l_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
    l_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
    
    r_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
    r_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
    r_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
    
    l_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
    l_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
    l_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
    
    r_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
    r_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
    r_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
       
    
    lh_angle=calculate_angle(l_shoulder,l_elbow,l_wrist)
    rh_angle=calculate_angle(r_shoulder,r_elbow,r_wrist)
    ll_angle=calculate_angle(l_hip,l_knee,l_ankle)
    rl_angle=calculate_angle(r_hip,r_knee,r_ankle)
    hip_angle= calculate_angle(l_shoulder,l_hip,l_knee)
    
    if 160<lh_angle<180:
        feed+='do namaskar '
    else:
        c+=1
        
    if 160<rh_angle<180:
        feed+='do namaskar '
    else:
        c+=1
        
    if 45<ll_angle<95 and 160<rl_angle<180:
        c+=1
    elif 45<rl_angle<95 and 160<ll_angle<180:
        c+=1
    else:
        feed+='Bend one leg '
        
    if 150<hip_angle<180:
        c+=1
    else:
        feed+='stand straight '
    if c>3:
        feed+='perfect'
    return feed

In [9]:
def warrior_feedback(landmarks):
    feed=''
    c=0
    l_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
    l_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
    l_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
    
    r_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
    r_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
    r_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
    
    l_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
    l_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
    l_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
    
    r_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
    r_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
    r_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
       
    
    ls_angle=calculate_angle(l_elabow,l_shoulder,l_hip)
    rs_angle=calculate_angle(r_elbow,r_shoulder,r_hip)
    ll_angle=calculate_angle(l_hip,l_knee,l_ankle)
    rl_angle=calculate_angle(r_hip,r_knee,r_ankle)
   
    if 80<ls_angle<100:
        c+=1
    else:
        feed+='adjust left hand straight '
        
    if 80<rs_angle<100:
        c+=1
    else:
        feed+='adjust right hand straight '
        
    if 80<ll_angle<100 and 140<rl_angle<180:
        c+=1
    elif 80<rl_angle<100 and 140<ll_angle<180:
        c+=1
    else:
        feed+='Bend one leg '
    
    if c>2:
        feed+='perfect'
    return feed

In [10]:
def gen_feedback(pose):
    feed_func=pose
    cap = cv2.VideoCapture(0)
    ## Setup mediapipe instance
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))

    size = (frame_width, frame_height)
    
#     #saving the video file
#     path=filename+'.avi'
#     result = cv2.VideoWriter(path,cv2.VideoWriter_fourcc(*'MJPG'),10, size)
    
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
        
            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Make detection
            results = pose.process(image)
    
            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
            # Extract landmarks
            landmarks = results.pose_landmarks.landmark
#             print(landmarks)

            # Visualize angle
            feedb=feed_func(landmarks)
            print(feedb)
            cv2.putText(image, str(feedb), (50,20), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,0,255), 2, cv2.LINE_AA)
            
            # Render detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
#             result.write(image)
            cv2.namedWindow('Mediapipe Feed', cv2.WINDOW_NORMAL)        
            cv2.imshow('Mediapipe Feed', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

In [11]:
def final_func1():
    TIMER = int(20)

    # Open the camera
    cap = cv2.VideoCapture(0)
    while True:
        # Read and display each frame
        ret, img = cap.read()
        cv2.imshow('pose_prediction', img)

        # check for the key pressed
        k = cv2.waitKey(125)

        # set the key for the countdown
        # to begin. Here we set q
        # if key pressed is q
        if k == ord('q'):
            prev = time.time()

            while TIMER >= 0:
                ret, img = cap.read()

                # Display countdown on each frame
                cv2.putText(img, str(TIMER),(200, 250), cv2.FONT_HERSHEY_SIMPLEX,7, (0, 255, 255),4, cv2.LINE_AA)
                cv2.imshow('pose_prediction', img)
                cv2.waitKey(125)

                # current time
                cur = time.time()

                # Update and keep track of Countdown, if time elapsed is one second than decrease the counter
                if cur-prev >= 1:
                    prev = cur
                    TIMER = TIMER-1

            else:
                ret, img = cap.read()

                # Display the clicked frame for 2 sec.You can increase time in waitKey also
                cv2.imshow('a', img)
                # time for which image displayed
                cv2.waitKey(2000)
                # Save the frame
                cv2.imwrite('capture.jpg', img)
                break
    # close the camera
    cap.release()
    cv2.destroyAllWindows()          
    
    img = Image.open("capture.jpg").convert('RGB').resize((256, 256), Image.ANTIALIAS)
    img = np.array(img)
    model = custom_model()
    model.load_weights(filepath='yoga_model_vgg19_dropout_2.h5')
    y=model.predict(img[None,:,:])
    pose=classes[np.argmax(y)]
    print("The predicted pose is",pose)
    
    if pose=='downdog':
        gen_feedback(downdog_feedback)
    elif pose=='goddess':
        gen_feedback(godess_feedback)
    elif pose=='plank':
        gen_feedback(plank_feedback)
    elif pose=='tree':
        gen_feedback(tree_feedback)
    elif pose=='warrior2':
        gen_feedback(warrior_feedback)

In [12]:
final_func1()

The predicted pose is goddess
adjust left hand adjust right hand adjust left leg adjust right leg 
adjust left hand adjust right hand adjust left leg adjust right leg 
adjust left hand adjust right hand adjust left leg adjust right leg 
adjust left hand adjust right hand adjust left leg adjust right leg 
adjust left hand adjust right hand adjust left leg adjust right leg 
adjust left hand adjust right hand adjust left leg adjust right leg 
adjust left hand adjust right hand adjust left leg adjust right leg 
adjust left hand adjust right hand adjust left leg adjust right leg 
adjust left hand adjust right hand adjust left leg adjust right leg 
adjust left hand adjust right hand adjust left leg adjust right leg 
adjust left hand adjust right hand adjust left leg adjust right leg 
adjust left hand adjust right hand adjust left leg adjust right leg 
adjust left hand adjust right hand adjust left leg adjust right leg 
adjust left hand adjust right hand adjust left leg adjust right leg 
adju

adjust left hand adjust right hand adjust left leg adjust right leg 
adjust left hand adjust right hand adjust left leg adjust right leg 
adjust right hand adjust left leg adjust right leg 
adjust right hand adjust left leg adjust right leg 
adjust right hand adjust left leg adjust right leg 
adjust left leg adjust right leg 
adjust left leg adjust right leg 
adjust left leg adjust right leg 
adjust left leg adjust right leg 
adjust left hand adjust left leg adjust right leg 
adjust left hand adjust left leg adjust right leg 
adjust left hand adjust left leg adjust right leg 
adjust left hand adjust left leg adjust right leg 
adjust left hand adjust left leg adjust right leg 
adjust left hand adjust left leg adjust right leg 
adjust left hand adjust left leg adjust right leg 
adjust left hand adjust left leg adjust right leg 
adjust left hand adjust left leg adjust right leg 
adjust left hand adjust left leg adjust right leg 
adjust left leg adjust right leg 
adjust left leg adjust rig

adjust left hand adjust left leg adjust right leg 
adjust left hand adjust left leg adjust right leg 
adjust left hand adjust left leg adjust right leg 
adjust left hand adjust left leg adjust right leg 
adjust left hand adjust left leg adjust right leg 
adjust left hand adjust left leg adjust right leg 
adjust left hand adjust left leg adjust right leg 
adjust left hand adjust left leg adjust right leg 
adjust left hand adjust left leg adjust right leg 
adjust left hand adjust left leg adjust right leg 
adjust left leg adjust right leg 
adjust left leg adjust right leg 
adjust left hand adjust left leg adjust right leg 
adjust left hand adjust left leg adjust right leg 
adjust left hand adjust left leg adjust right leg 
adjust left hand adjust left leg adjust right leg 
adjust left hand adjust left leg adjust right leg 
adjust left hand adjust left leg adjust right leg 
adjust left hand adjust left leg adjust right leg 
adjust left hand adjust left leg adjust right leg 
adjust left ha